<a href="https://colab.research.google.com/github/peremartra/Large-Language-Model-Notebooks-Course/blob/main/5-Fine%20Tuning/Prompt_Tuning_PEFT-V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<div align="center">
<h1><a href="https://github.com/peremartra/Large-Language-Model-Notebooks-Course">LLM Hands On Course</a></h1>
    <h3>Understand And Apply Large Language Models</h3>
    <h2>Introduction to Prompt Tuning using PEFT from Hugging Face.</h2>
    <h3>Fine-tune a Foundational Model effortless</h3>
    <h4>Revisioned with new samples April 2024</h4>
    <p>by <b>Pere Martra</b></p>
    
</div>

<br>

<div align="center">
    &nbsp;
    <a target="_blank" href="https://www.linkedin.com/in/pere-martra/"><img src="https://img.shields.io/badge/style--5eba00.svg?label=LinkedIn&logo=linkedin&style=social"></a>
    
</div>

<br>
<hr>

# Prompt Tuning

## Brief introduction to Prompt Tuning.
It’s an Additive Fine-Tuning technique for models. This means that we WILL NOT MODIFY ANY WEIGHTS OF THE ORIGINAL MODEL. You might be wondering, how are we going to perform fine-tuning then? Well, we will train additional layers that are added to the model. That’s why it’s called an Additive technique.

Considering it’s an Additive technique and its name is Prompt-Tuning, it seems clear that the layers we’re going to add and train are related to the prompt.

![My Image](https://github.com/peremartra/Large-Language-Model-Notebooks-Course/blob/main/img/Martra_Figure_5_Prompt_Tuning.jpg?raw=true)

We are creating a type of superprompt by enabling a model to enhance a portion of the prompt with its acquired knowledge. However, that particular section of the prompt cannot be translated into natural language. **It's as if we've mastered expressing ourselves in embeddings and generating highly effective prompts.**

In each training cycle, the only weights that can be modified to minimize the loss function are those integrated into the prompt.

The primary consequence of this technique is that the number of parameters to train is genuinely small. However, we encounter a second, perhaps more significant consequence, namely that, **since we do not modify the weights of the pretrained model, it does not alter its behavior or forget any information it has previously learned.**

The training is faster and more cost-effective. Moreover, we can train various models, and during inference time, we only need to load one foundational model along with the new smaller trained models because the weights of the original model have not been altered

## What are we going to do in the notebook?
We are going to train two different models using two datasets, each with just one pre-trained model from the Bloom family. One will be trained to generate prompts and the other to detect hate in sentences.

Additionally, we'll explore how to load both models with only one copy of the foundational model in memory.


## Loading the Peft Library
This library contains the Hugging Face implementation of various fine-tuning techniques, including Prompt Tuning

In [1]:
!pip install -q peft==0.10.0
!pip install -q datasets==2.18.0
!pip install -q accelerate==0.29.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.8 MB/s eta 0:00:00


From the transformers library, we import the necessary classes to instantiate the model and the tokenizer.

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForSeq2SeqLM

## Loading the model and the tokenizers.

Bloom is one of the smallest and smartest models available for training with the PEFT Library using Prompt Tuning.

I'm opting for the smallest one to minimize training time and avoid memory issues in Colab. Feel Free to try with a bigger one if you have acces to a good GPU.

In [3]:
model_name = "bigscience/bloomz-560m"
#model_name = "bigscience/bloomz-1b1"
NUM_VIRTUAL_TOKENS = 20
#If you just want to test the solution, you can reduce the EPOCHs.
NUM_EPOCHS_PROMPT = 50
NUM_EPOCHS_CLASSIFIER = 10
device = "cuda" #Replace with "mps" for Silicon chips.

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
foundational_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map = device
)

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/715 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

## Inference with the pre trained bloom model



In [5]:
#this function returns the outputs from the model received, and inputs.
def get_outputs(model, inputs, max_new_tokens=100):
    outputs = model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_new_tokens=max_new_tokens,
        #temperature=0.2,
        #top_p=0.95,
        #do_sample=True,
        repetition_penalty=1.5, #Avoid repetition.
        early_stopping=True, #The model can stop before reach the max_length
        eos_token_id=tokenizer.eos_token_id
    )
    return outputs

To compare the pre-trained model with the same model after the prompt-tuning process, I will run the same sentence on both models.

Since I'm creating a model that can generate prompts, I'll instruct it to provide a prompt that makes it act like a fitness trainer.

In [6]:
input_prompt = tokenizer("Act as a fitness Trainer. Prompt: ", return_tensors="pt")
foundational_outputs_prompt = get_outputs(foundational_model,
                                          input_prompt.to(device),
                                          max_new_tokens=50)

print(tokenizer.batch_decode(foundational_outputs_prompt, skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


['Act as a fitness Trainer. Prompt:  Follow up with your trainer']


The model doesn't know what its mission is and answers as best as it can. It's not a bad response, but it's not what we're looking for.

# Prompt Creator
## Preparing Datasets
The Dataset used, for this first example, is:
* https://huggingface.co/datasets/fka/awesome-chatgpt-prompts



In [7]:
import os
from datasets import load_dataset

In [8]:
dataset_prompt = "fka/awesome-chatgpt-prompts"

In [9]:
def concatenate_columns_prompt(dataset):
    def concatenate(example):
        example['prompt'] = "Act as a {}. Prompt: {}".format(example['act'], example['prompt'])
        return example

    dataset = dataset.map(concatenate)
    return dataset

In [10]:
#Create the Dataset to create prompts.
data_prompt = load_dataset(dataset_prompt)
data_prompt['train'] = concatenate_columns_prompt(data_prompt['train'])

data_prompt = data_prompt.map(lambda samples: tokenizer(samples["prompt"]), batched=True)
train_sample_prompt = data_prompt["train"].remove_columns('act')

Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/153 [00:00<?, ? examples/s]

Map:   0%|          | 0/153 [00:00<?, ? examples/s]

In [11]:
print(train_sample_prompt)

Dataset({
    features: ['prompt', 'input_ids', 'attention_mask'],
    num_rows: 153
})


In [12]:
print(train_sample_prompt[:2])

{'prompt': ['Act as a Linux Terminal. Prompt: I want you to act as a linux terminal. I will type commands and you will reply with what the terminal should show. I want you to only reply with the terminal output inside one unique code block, and nothing else. do not write explanations. do not type commands unless I instruct you to do so. when i need to tell you something in english, i will do so by putting text inside curly brackets {like this}. my first command is pwd', 'Act as a English Translator and Improver. Prompt: I want you to act as an English translator, spelling corrector and improver. I will speak to you in any language and you will detect the language, translate it and answer in the corrected and improved version of my text, in English. I want you to replace my simplified A0-level words and sentences with more beautiful and elegant, upper level English words and sentences. Keep the meaning same, but make them more literary. I want you to only reply the correction, the impro

## prompt-tuning configuration.  

API docs:
https://huggingface.co/docs/peft/main/en/package_reference/tuners#peft.PromptTuningConfig


In [13]:
from peft import  get_peft_model, PromptTuningConfig, TaskType, PromptTuningInit

generation_config_prompt = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM, #This type indicates the model will generate text.
    prompt_tuning_init=PromptTuningInit.RANDOM,  #The added virtual tokens are initializad with random numbers
    num_virtual_tokens=NUM_VIRTUAL_TOKENS, #Number of virtual tokens to be added and trained.
    tokenizer_name_or_path=model_name #The pre-trained model.
)


We will create two  prompt tuning models using the same pre-trained model and the same config, but with a different Dataset.

In [14]:
peft_model_prompt = get_peft_model(foundational_model,
                                   generation_config_prompt)
print(peft_model_prompt.print_trainable_parameters())

trainable params: 20,480 || all params: 559,235,072 || trainable%: 0.0036621451381361144
None


**That's amazing: did you see the reduction in trainable parameters? We are going to train a 0.001% of the paramaters available.**

Now we are going to create the training arguments, and we will use the same configuration in both trainings.

In [16]:

import os

working_dir = "./"

#Is best to store the models in separate folders.
#Create the name of the directories where to store the models.
output_directory_prompt =  os.path.join(working_dir, "peft_outputs_prompt")
output_directory_classifier =  os.path.join(working_dir, "peft_outputs_classifier")

#Just creating the directoris if not exist.
if not os.path.exists(working_dir):
    os.mkdir(working_dir)
if not os.path.exists(output_directory_prompt):
    os.mkdir(output_directory_prompt)


In [15]:
from transformers import TrainingArguments
def create_training_arguments(path, learning_rate=0.0035, epochs=6, autobatch=True):
    training_args = TrainingArguments(
        output_dir=path, # Where the model predictions and checkpoints will be written
        #use_cpu=True, # This is necessary for CPU clusters.
        auto_find_batch_size=autobatch, # Find a suitable batch size that will fit into memory automatically
        learning_rate= learning_rate, # Higher learning rate than full fine-tuning
        #per_device_train_batch_size=4,
        num_train_epochs=epochs
    )
    return training_args

We need to indicate the directory containing the model when creating the TrainingArguments.

## Training first model

We will create the trainer Object, one for each model to train.  

In [17]:
training_args_prompt = create_training_arguments(
    output_directory_prompt,
    3e-2,
    NUM_EPOCHS_PROMPT)

In [18]:
from transformers import Trainer, DataCollatorForLanguageModeling
def create_trainer(model, training_args, train_dataset):
    trainer = Trainer(
        # We pass in the PEFT version of the foundation model, bloomz-560M
        model=model,
        #The args for the training.
        args=training_args,
        train_dataset=train_dataset, #The dataset used to train the model.
        # mlm=False indicates not to use masked language modeling
        data_collator=DataCollatorForLanguageModeling(
            tokenizer,
            mlm=False)
    )
    return trainer


In [19]:
#Training first model.
trainer_prompt = create_trainer(peft_model_prompt,
                                training_args_prompt,
                                train_sample_prompt)
trainer_prompt.train()

Step,Training Loss
500,2.615100
1000,2.391800


TrainOutput(global_step=1000, training_loss=2.5034515380859377, metrics={'train_runtime': 601.7559, 'train_samples_per_second': 12.713, 'train_steps_per_second': 1.662, 'total_flos': 2240433048453120.0, 'train_loss': 2.5034515380859377, 'epoch': 50.0})

Release GPU memory.

In [20]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()

139

## Save model
We are going to save the model. These models are ready to be used, as long as we have the pre-trained model from which they were created in memory.

In [21]:
trainer_prompt.model.save_pretrained(output_directory_prompt)

## Inference first tuned model

You can load the model from the path that you have saved to before, and ask the model to generate text based on our input before!

In [22]:
from peft import PeftModel

loaded_model_peft = PeftModel.from_pretrained(foundational_model,
                                         output_directory_prompt,
                                         #device_map=device,
                                         is_trainable=False)

In [23]:
loaded_model_prompt_outputs = get_outputs(loaded_model_peft,
                                          input_prompt,
                                          max_new_tokens=50)
print(tokenizer.batch_decode(loaded_model_prompt_outputs,
                             skip_special_tokens=True))

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


['Act as a fitness Trainer. Prompt: 帮我想找个健身教练。 I want you to act like an expert in your field and explain the basics of different types, styles & exercises for me.  My first exercise is cardio training with weights or elliptical machines; my second one will be strength']


'Act as a fitness Trainer. Prompt: ive got to do something with my life and i want you help me find the right way of doing it, so that I can get rid from this problem in no time! - Comments (0)'



Let's compare the result of the model before and after being fine-tuned with prompt-tuning.

**Input for the model**
```
Act as a fitness Trainer. Prompt:
```

**Original model**
```
Act as a fitness Trainer. Prompt:  Follow up with your trainer
```
**Trained for classification with Prompt-tuning 50 Epochs & 8 virtual tokens:**
```
Act as a fitness Trainer. Prompt: ＋ Acts like an expert in the field of sports and health, but does not provide detailed information about his work or products to help you understand them better.  + I want my first client referred me through this website for their gym membership program which is based on physical activity training exercises that are easy enough (eight minutes) per week with no need any special equipment required.   - First Question : What would be your role?
```

It's very clear that the result is quite different, it's not exactly what we're looking for but it's much closer.

It's possible that we're at the limit of what Bloom's smallest model can offer. Try with any other model, surely with the one with 1B parameters the result will be better.

# Hate Classifier
##Loading the Dataset

* https://huggingface.co/datasets/SetFit/ethos_binary

In [24]:
input_classifier = tokenizer("Sentence : Head is the shape of a light bulb. Label : ", return_tensors="pt")
foundational_outputs_prompt = get_outputs(foundational_model,
                                          input_classifier.to(device),
                                          max_new_tokens=50)

print(tokenizer.batch_decode(foundational_outputs_prompt, skip_special_tokens=True))

['Sentence : Head is the shape of a light bulb. Label :  head']


The model has no idea what its purpose is, so it completes the sentence as best as it can.

In [25]:
dataset_classifier = "SetFit/ethos_binary"

def concatenate_columns_classifier(dataset):
    def concatenate(example):
        example['text'] = "Sentence : {} Label : {}".format(example['text'], example['label_text'])
        return example

    dataset = dataset.map(concatenate)
    return dataset

In [26]:
data_classifier = load_dataset(dataset_classifier)
data_classifier['train'] = concatenate_columns_classifier(data_classifier['train'])

data_classifier = data_classifier.map(lambda samples: tokenizer(samples["text"]), batched=True)
train_sample_classifier = data_classifier["train"].remove_columns(['label', 'label_text', 'text'])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/repocard.py:105: UserWarning: Repo card metadata block was not found. Setting CardData to empty.
  warnings.warn("Repo card metadata block was not found. Setting CardData to empty.")


Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/598 [00:00<?, ? examples/s]

Map:   0%|          | 0/598 [00:00<?, ? examples/s]

Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [27]:
data_classifier

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 598
    })
    test: Dataset({
        features: ['text', 'label', 'label_text', 'input_ids', 'attention_mask'],
        num_rows: 400
    })
})

In [28]:
train_sample_classifier

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 598
})

I have deleted all the columns from the dataset that are not strictly necessary for training, that is to say, I have removed all columns that are not essential for the model's learning process.

In [29]:
print(train_sample_classifier[1:2])

{'input_ids': [[62121, 1671, 915, 473, 760, 10190, 513, 16154, 60, 19821, 138929, 20812, 426, 18833, 18816, 75536, 45617, 39469, 19368, 17956, 57274, 3758, 18065, 38, 44140, 17956, 72870, 8309, 9492, 15, 614, 156801, 85061, 48283, 44419, 426, 16472, 96789, 602, 45227, 43111, 181485, 435, 19821, 60, 48283, 44419, 426, 16472, 96789, 614, 156801, 77658, 915, 74549, 40423]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


## prompt-tuning configuration

In [30]:
generation_config_classifier = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM, #This type indicates the model will generate text.
    prompt_tuning_init=PromptTuningInit.TEXT,  #
    prompt_tuning_init_text="Indicates whether the sentence contains hate speech or not",
    num_virtual_tokens=NUM_VIRTUAL_TOKENS, #Number of virtual tokens to be added and trained.
    tokenizer_name_or_path=model_name #The pre-trained model.
)

In [31]:
peft_model_classifier = get_peft_model(foundational_model, generation_config_classifier)
print(peft_model_classifier.print_trainable_parameters())

trainable params: 20,480 || all params: 559,235,072 || trainable%: 0.0036621451381361144
None


In [32]:
if not os.path.exists(output_directory_classifier):
    os.mkdir(output_directory_classifier)

In [33]:
training_args_classifier = create_training_arguments(output_directory_classifier,
                                                    3e-2,
                                                    NUM_EPOCHS_CLASSIFIER)

## Training Second Model

In [34]:
trainer_classifier = create_trainer(peft_model_classifier,
                                   training_args_classifier,
                                   train_sample_classifier)
trainer_classifier.train()

Step,Training Loss


Step,Training Loss


Step,Training Loss
500,3.312300
1000,3.273400
1500,3.242900
2000,3.201300
2500,3.197500


TrainOutput(global_step=2990, training_loss=3.2301716578046613, metrics={'train_runtime': 233.9847, 'train_samples_per_second': 25.557, 'train_steps_per_second': 12.779, 'total_flos': 667918083244032.0, 'train_loss': 3.2301716578046613, 'epoch': 10.0})

In [35]:
trainer_classifier.model.save_pretrained(output_directory_classifier)

## Inference second Model

In [36]:
loaded_model_peft.load_adapter(output_directory_classifier, adapter_name="classifier")
loaded_model_peft.set_adapter("classifier")

In [40]:
loaded_model_sentences_outputs = get_outputs(loaded_model_peft,
                                             input_classifier, max_new_tokens=3)
print(tokenizer.batch_decode(loaded_model_sentences_outputs, skip_special_tokens=True))

["Sentence : I don't like short people, no idea why they exist. Label :  hate speech label"]


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:535: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Let's check how the model's response has changed with training:

**Input for the model**
```
Sentence : Head is the shape of a light bulb. Label :
Sentence : I don't liky short people, no idea why they exist. Label :
```

**Original model**
```
Sentence : Head is the shape of a light bulb. Label :  head
Sentence : I don't liky short people, no idea why they exist. Label :  No
```
**Trained for classification with Prompt-tuning 50 epochs & 8 virtual tokens**
```
Sentence : Head is the shape of a light bulb. Label :  no hate speech
Sentence : I don't liky short people, no idea why they exist. Label :  hate speech
```

**Trained for classification with Prompt-tuning 10 epochs & 20 virtual tokens**
```
Sentence : Head is the shape of a light bulb. Label :  no hate speech
Sentence : I don't liky short people, no idea why they exist. Label :  hate speech
```
It's clear that the training has fulfilled its purpose. The original model doesn't know what its mission is and tries to complete the sentences as best as it can. On the other hand, the updated model with prompt-tuning does know what its mission is and is able to classify the sentences correctly and in the indicated format.


# Conclusion
Prompt Tuning is an amazing technique that can save us hours of training and a significant amount of money. In the notebook, we have trained two models in just a few minutes, and we can have both models in memory, providing service to different clients.

If you want to try different combinations and models, the notebook is ready to use another model from the Bloom family.

You can change the number of epochs to train, the number of virtual tokens, and the model. However, there are many configurations to change.

*The responses of the fine-tuned models may vary every time we train them. I've pasted the results of one of my trainings, but the actual results may differ.*